In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("wine-raitngs.csv")
df.head()

,name,region,variety,rating,notes
0,1000 Stories Bourbon Barrel Aged Batch Blue Ca...,"Mendocino, California",Red Wine,91,"This is a very special, limited release of 100..."
1,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,Red Wine,89,The California Gold Rush was a period of coura...
2,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,Red Wine,90,The California Gold Rush was a period of coura...
3,1000 Stories Bourbon Barrel Aged Zinfandel 2013,"North Coast, California",Red Wine,91,"The wine has a deep, rich purple color. An int..."
4,1000 Stories Bourbon Barrel Aged Zinfandel 2014,California,Red Wine,90,Batch #004 is the first release of the 2014 vi...


In [3]:
df.describe()

,rating
count,32980.000000
mean,91.186113
std,2.190520
min,85.000000
25%,90.000000
50%,91.000000
75%,92.000000
max,99.000000


In [4]:
df = df[df['variety'].notna()]

In [5]:
data = df.sample(1000).to_dict('records')
len(data)

1000

In [6]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

c:\Users\MSI\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant Instance

In [9]:
# Create collection to store books
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

C:\Users\MSI\AppData\Local\Temp\ipykernel_10328\1507606962.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [10]:
# Vectorize
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [11]:
# Search Time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode("A wine from Mendoza Argentina").tolist(),
    limit=3
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'name': 'Arboleda Cabernet Sauvignon 1999', 'region': 'Chile', 'variety': 'Red Wine', 'rating': 85, 'notes': "Beautifully balanced and complex, our Arboleda 1999 Cabernet Sauvignon has lush blackberry, black currant and rosehip jam aromas. Intriguing nuances of chocolate and mint mingle with the rich, dark fruit character. The intense fruitiness continues through the flavors, supported by round, velvety tannins. Subtle notes of tobacco and sun-dried fruit linger on the elegant finish. The excellent structure of this wine reflects grapes grown on our Los Morros Estate in Chile's highly acclaimed Maipo Valley. Gentle handcrafting and 18 months of barrel aging, at our state-of-the-art Arboleda Winery, enhanced the velvety character. Our new Arboleda collection of wines showcases the personalities of Chile's finest appellations."} score: 0.6501171029719838
{'name': 'Bodega DiamAndes Uco Malbec 2011', 'region': 'Uco Valley, Mendoza, Argentina', 'variety': 'Red Wine', 'rating': 92, 'notes':

C:\Users\MSI\AppData\Local\Temp\ipykernel_10328\3246915765.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [12]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [13]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'name': 'Elsa Bianchi Malbec 2014', 'region': 'Mendoza, Argentina', 'variety': 'Red Wine', 'rating': 89, 'notes': 'This Malbec is red-violet in color, with aromas of plums and black cherries and a subtle, smoky scent. The beauty of Malbec in Argentina is its ability to combine a rich, weighty mouth-feel with a soft silkiness normally associated with lighter wines. Elsa Malbec takes the promise of the nose through to the palate, with dark fruits that mimics the aroma. The soft, supple palate and smooth tannins lead to a round and harmonious finish.'} score: 0.7020514805788002
{'name': "Casarena Jamilla's Vineyard Malbec 2011", 'region': 'Mendoza, Argentina', 'variety': 'Red Wine', 'rating': 91, 'notes': "Casarena Jamilla's Vineyard Malbec is pure elegance. Rocky limestone and shallow soil terroir's best expression is  this wine with mineral notes, floral and red fruit flavors. Fresh. "} score: 0.6493623228466596
{'name': 'Bodega Colome Altura Maxima Malbec 2012', 'region': 'Salta, Arge

C:\Users\MSI\AppData\Local\Temp\ipykernel_10328\1255892075.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [15]:
# Define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [ ]:
# Connecting to a large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1",
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model= "LlaMA_CPP",
    messages=[
        {"role": "system", "content": "You are a chatbot, a wine specialist. Your top priority is to "},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)

print(completion.choices[0].message)